In [4]:
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import fsspec

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12,8

# !pip install opencv-python-headless
import cv2
from matplotlib.patches import Rectangle
from typing import List, Tuple
import itertools
from copy import copy, deepcopy
import os
import shutil

import getpass
import azure.storage.blob
from azure.storage.blob import BlobClient, BlobServiceClient
from azure.core.exceptions import ResourceExistsError


In [8]:
####################################
# Read dataset
####################################

# import heat extremes data
fn = f"Ext_max_t__Rgn_1__2015__Abv_Avg_5_K_for_4_days__CMIP6_ssp245_Avg_yrs_1950_79.nc"
ds_heat = xr.open_mfdataset(fn)

print(ds_heat.attrs['Number of continuous days to be considered extreme'], "days")
print(ds_heat.attrs['threshold'], "K above")

4 days
5 K above


In [73]:
ds_heat

<xarray.Dataset>
Dimensions:          (time: 365, lat: 284, lon: 644)
Coordinates:
  * time             (time) object 2015-01-01 12:00:00 ... 2015-12-31 12:00:00
  * lat              (lat) float64 14.12 14.38 14.62 14.88 ... 84.38 84.62 84.88
  * lon              (lon) float64 190.1 190.4 190.6 190.9 ... 350.4 350.6 350.9
Data variables:
    tasmax           (time, lat, lon) float32 dask.array<chunksize=(365, 284, 644), meta=np.ndarray>
    above_threshold  (time, lat, lon) float32 dask.array<chunksize=(365, 284, 644), meta=np.ndarray>
    extreme_yn       (time, lat, lon) float64 dask.array<chunksize=(365, 284, 644), meta=np.ndarray>
Attributes: (12/24)
    Dataset:                                             1-year Extreme max T...
    About dataset:                                       1-year Extreme max T...
    Data variables:                                      tasmax, above_thresh...
    Data description:                                    max temperature; dif...
    Range:                                               1 year
    Analysis year:                                       2015
    ...                                                  ...
    region_bottom_lat:                                   14
    region_left_lon:                                     190
    region_right_lon:                                    350
    region_img_url:                                      https://github.com/t...
    Years processed:                                     1
    Store as:                                            Ext_max_t__Rgn_1__20...

In [84]:
####################################
# PROCESS: BOUNDING-BOXES ALGORITHM to extract metadata
####################################

"""
Bounding-boxes algorithm to find the extend of the heat events and
produce visualizations. It uses the the heat events y/n dataset 
which was (supposed to be pre-) produced by the "Heatwave Analysis" algorithm. 
"""

def bounding_boxes(arr2d: np.array) -> List[tuple]:
    
    H = arr2d.astype(np.uint8)
    ret, thresh = cv2.threshold(H, 0, 1, 0, cv2.THRESH_BINARY)
    contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = [cv2.boundingRect(c) for c in contours]
    return boxes

def isoverlap(box1:tuple, box2:tuple) -> bool:
    """Return True if two windows overlap"""
    x1,y1,w1,h1 = box1
    x2,y2,w2,h2 = box2
    return not (x2>x1+w1 or x2+w2<x1 or y2>y1+h1 or y2+h2<y1)

def outer(box1:tuple, box2:tuple) -> tuple:
    """Fuse two windows into one, parent window."""
    x1,y1,w1,h1 = box1
    x2,y2,w2,h2 = box2
    x = min(x1,x2)
    y = min(y1,y2)
    w = max(x1+w1,x2+w2)-x
    h = max(y1+h1,y2+h2)-y
    return (x, y, w, h)

def istiny(box:tuple, min_area:int) -> bool:
    x,y,w,h = box
    return w*h <= min_area

def filter_tiny_ones(boxes:List[tuple]) -> List[tuple]:
    return [c for c in boxes if not istiny(c, 10)]

def collapse(boxes:List[tuple]) -> List[tuple]:
    
    for box1, box2 in itertools.combinations(boxes, 2):
        if isoverlap(box1,box2):
            boxes.remove(box1)
            boxes.remove(box2)
            boxes.append(outer(box1,box2))
            return collapse(boxes) # recursion

    boxes.sort(key=lambda _:_[0])
    return boxes

def array2boxes(arr2d:np.array) -> List[tuple]:
    """Pipeline. Takes a time-slice (2D array) and returns the boxes."""
    boxes = bounding_boxes(arr2d)
    boxes = filter_tiny_ones(boxes)
    boxes = collapse(boxes)
    return boxes

def groupby_heat_events(arr3d:np.array) -> pd.DataFrame:
    
    rows = []
    num_days = arr3d.shape[0]

    for i in range(num_days):
        arr2d = arr3d[i,:,:]
        boxes = array2boxes(arr2d)
        rows += [dict(time=i, boxes=boxes)]
        
    df = pd.DataFrame(rows)
    df['hasEvent'] = df['boxes'].apply(lambda x: len(x)) > 0
    df['label'] = df['hasEvent'].diff().ne(False).cumsum()

    dff = df[df['hasEvent']]

    dfg = dff.groupby('label').agg({
        'time':[np.min,np.max], 
        'boxes':lambda _: collapse(np.sum(_))
    }).reset_index()
    
    dfg.columns = ['label', 'i1', 'i2', 'boxes']
    dfg = dfg.drop('label', axis=1)
    
    return dfg

def box_events(ds:xr.Dataset, summer_only=True) -> pd.DataFrame:
    
    # accept 1-year data only
    n_days = len(ds_heat['time'])
    if n_days != 365:
        err = f"dimension mismatch: expected 365 but got {n_days}"
        raise ValueError(err)
    
    # subset for summer months
    da = ds_heat['extreme_yn']
    
    if summer_only:
        sm = [5, 6, 7, 8, 9]
        arr3d = da.loc[da['time.month'].isin(sm)].values
    else:
        arr3d = da.values
    
    # find bounding boxes
    df = groupby_heat_events(arr3d)
    
    # add dates to the df
    dr = pd.DatetimeIndex(ds['time'].dt.floor('D').values.astype('str'))
    df = df.assign(d1=dr[df['i1']], d2=dr[df['i2']])
    
    return df

box_events(ds_heat)


,i1,i2,boxes,d1,d2
0,0,152,"[(14, 0, 579, 279), (597, 200, 14, 9), (598, 2...",2015-01-01,2015-06-02


In [20]:
arr3d = ds_heat['extreme_yn'].values

rows = []
num_days = arr3d.shape[0]
for i in range(num_days):
    arr2d = arr3d[i,:,:]
    boxes = array2boxes(arr2d)
    rows += [dict(time=i, boxes=boxes)]
df = pd.DataFrame(rows)
df['hasEvent'] = df['boxes'].apply(lambda x: len(x)) > 0
df['label'] = df['hasEvent'].diff().ne(False).cumsum()
    
df

,time,boxes,hasEvent,label
0,0,"[(7, 178, 98, 42), (204, 176, 14, 5), (280, 12...",True,1
1,1,"[(7, 178, 105, 42), (175, 160, 54, 31), (280, ...",True,1
2,2,"[(7, 178, 151, 45), (175, 160, 54, 31), (272, ...",True,1
3,3,"[(7, 178, 152, 46), (175, 160, 54, 31), (176, ...",True,1
4,4,"[(22, 178, 137, 46), (175, 160, 54, 31), (176,...",True,1
...,...,...,...,...
360,360,"[(0, 195, 6, 3), (7, 125, 288, 90), (359, 155,...",True,1
361,361,"[(0, 195, 6, 3), (7, 125, 288, 101), (359, 154...",True,1
362,362,"[(0, 195, 6, 3), (7, 180, 127, 46), (143, 177,...",True,1
363,363,"[(0, 195, 6, 3), (7, 180, 120, 46), (143, 177,...",True,1


In [70]:

dr = pd.DatetimeIndex(ds_heat['time'].dt.floor('D').values.astype('str'))

dff = df[df['hasEvent']]
dfg = dff.groupby('label').agg({
    'time':[np.min,np.max], 
    'boxes':lambda _: collapse(np.sum(_))
}).reset_index()
dfg.columns = ['label', 'i1', 'i2', 'boxes']
dfg = dfg.assign(d1=dr[dfg['i1']], d2=dr[dfg['i2']])
dfg = dfg.drop('label', axis=1)
dfg

,i1,i2,boxes,d1,d2
0,0,364,"[(0, 195, 6, 3), (7, 0, 637, 279)]",2015-01-01,2015-12-31


DatetimeIndex(['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04',
               '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08',
               '2015-01-09', '2015-01-10',
               ...
               '2015-12-22', '2015-12-23', '2015-12-24', '2015-12-25',
               '2015-12-26', '2015-12-27', '2015-12-28', '2015-12-29',
               '2015-12-30', '2015-12-31'],
              dtype='datetime64[ns]', length=365, freq=None)

,i1,i2,boxes,d1,d2
0,133,140,"[(56, 121, 90, 75)]",1971-05-14,1971-05-21
1,155,162,"[(134, 177, 5, 5), (216, 160, 86, 40)]",1971-06-05,1971-06-12
2,167,169,"[(158, 165, 12, 7)]",1971-06-17,1971-06-19
3,175,180,"[(70, 125, 31, 29), (85, 157, 5, 4), (91, 185,...",1971-06-25,1971-06-30
4,182,195,"[(104, 155, 58, 30), (129, 103, 78, 47)]",1971-07-02,1971-07-15
5,203,206,"[(219, 160, 43, 15)]",1971-07-23,1971-07-26
6,210,222,"[(64, 167, 15, 8), (148, 152, 96, 33), (251, 1...",1971-07-30,1971-08-11
7,224,226,"[(127, 101, 7, 2)]",1971-08-13,1971-08-15
8,228,247,"[(141, 116, 102, 67)]",1971-08-17,1971-09-05
9,249,253,"[(135, 102, 15, 24), (192, 72, 10, 5)]",1971-09-07,1971-09-11


In [ ]:
####################################
# PROCESS: Generate Images of detected-heat events
####################################

# # temp data
# ds = xr.open_mfdataset('Koray/CMIP5/*.nc')

# # Ravi's avg 
# ds_avg = xr.open_dataset("Koray/Avg_temp_max_CMIP5__30_yrs__1950_to_1979.nc")


for year in range(1970,1980):

    dr = pd.date_range(start=f'1/1/{year}', periods=365, freq='D').date
    arr3d = np.load(f'Koray/CMIP5_flagged/arr_heat3d-{year}.npy') # must be ready
    df_events = groupby_heat_events(arr3d)

    for ev, (i1, i2) in df_events[['i1','i2']].iterrows():
        # to write the image to disk
        folder_name = f"{year}-event{str(ev).zfill(2)}"
        folder_path = f"Koray/CMIP5_images/{folder_name}"
        if not os.path.exists(folder_path):
            os.mkdir(folder_path)
        
        # loop through heat events, some are 3 days long, some are 20
        for i, idx in enumerate(range(i1,i2+1)):

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,4))

            day = dr[idx].strftime("%Y-%m-%d")
            tasmax = ds['tasmax'].sel(time=day, **area)
            tavg3d = ds_avg['tasmaxavg'].sel(day=idx, **area)
            tdiff = tasmax - tavg3d

            im1 = tdiff.squeeze().plot.imshow(ax=ax1, cmap='Reds', vmin=-4, vmax=4)
            im2 = (tasmax-273.15).squeeze().plot.imshow(ax=ax2, vmin=5, vmax=50)

            # add bounding boxes
            boxes = df_events['boxes'].iloc[ev]
            for b in boxes:
                x,y,w,h = b
                x = float(tdiff.coords['lon'][x])
                y = float(tdiff.coords['lat'][y])
                _, leny, lenx = tdiff.shape

                w = w * (lon_max-lon_min) / lenx
                h = h * (lat_max-lat_min) / leny 

                rect = Rectangle((x, y), w, h, color='b', fill=False, linewidth=2)
                ax1.add_patch(rect)
                rect = Rectangle((x, y), w, h, color='b', fill=False, linewidth=2)
                ax2.add_patch(rect)

            fig.tight_layout()

            # save each image, each event has multiple images, one per event day.
            filename = f"{folder_path}/{day}.png"
            fig.savefig(filename, dpi=fig.dpi)
            plt.close(fig)

            # upload image to azure
            sas_url = f"{url_prefix}/{blob_folder}/{filename}?{sas_token}"
            blob_client = BlobClient.from_blob_url(sas_url)
            with open(filename, "rb") as f:
                try:
                    blob_client.upload_blob(f)
                except ResourceExistsError:
                    pass
    
        # flush VM disk space
        shutil.rmtree(folder_path)

